## Google spreadsheet crosswalk downloader

This notebook helps maintain a list of google spreadsheet crosswalks that are downloaded as xlsx files and converted for upload in json-ld format. This makes it easier for collaboratively edited/maintained crosswalk files to be updated.

To do:

0. ~~Enable reading of google spreadsheets similar to xlsx file~~
1. ~~Create list of crosswalks~~
2. Enable checking for google spreadsheet in list of crosswalks and adding it if it's not there
3. Download and save google spreadsheet as an xlsx file
4. Create GitHub pull request template
5. Create script for maintaining list (that can run in conjunction with a GitHub pull request template and GitHub action)


In [22]:
import os
import json
import pandas as pd
import requests
from datetime import datetime
import openpyxl
from io import BytesIO as BytesIO

nb_path = os.getcwd()
parent_path = os.path.dirname(nb_path)
tempfiles = os.path.join(parent_path,'tempfiles')
print(parent_path)

C:\Users\gtsueng\Anaconda3\envs\outbreak\DDE-CrossWalks


In [4]:
#### Handle spreadsheets from google
def parse_g_sheet_url(gsheeturl):
    baseurl = 'https://docs.google.com/spreadsheets/d/'
    tmpurl = gsheeturl.replace(baseurl,'')
    tmpurlcontent = tmpurl.split('/')
    spreadsheetId = tmpurlcontent[0]
    return spreadsheetId

def load_g_cred(parent_path):
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from pydrive2.auth import ServiceAccountCredentials
    gauth = GoogleAuth()
    scope = ['https://www.googleapis.com/auth/drive']
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(os.path.join(parent_path,'credentials.json'), scope)
    return gauth

def load_g_sheet_data(parent_path, gsheeturl):
    gauth = load_g_cred(parent_path)
    spreadsheetID = parse_g_sheet_url(gsheeturl)
    url = f"https://docs.google.com/spreadsheets/export?id={spreadsheetID}&exportFormat=xlsx"
    res = requests.get(url, headers={"Authorization": "Bearer " + str(gauth.attr['credentials'].access_token)})
    data_file = res.content
    return data_file

#### Handle spreadsheets saved in GitHub
def load_github_data(githuburl):
    url = f"{githuburl}?raw=true"
    res = requests.get(url)
    data_file = res.content
    return data_file

#### Handle spreadsheets saved elsewhere
def load_xls_data(otherurl):
    res = requests.get(url)
    data_file = res.content
    return data_File

#### Pull basic metadata from spreadsheet
def get_xwalk_meta(parent_path, inputurl):
    if 'github' in inputurl:
        data_file = load_github_data(inputurl)
        xwalk_dict = {'url':inputurl, 'urlType':'GitHub'}
    elif 'google' in inputurl:
        data_file = load_g_sheet_data(parent_path, inputurl)
        xwalk_dict = {'url':inputurl, 'urlType':'Gsheet'}
    else:
        try:
            data_file = load_xls_data(otherurl)
        except:
            data_file = None
        xwalk_dict = {'url':inputurl, 'urlType':'Other'}
    data_stream = BytesIO(data_file)
    values = pd.read_excel(data_stream,sheet_name='metainfo',header=0,index_col=None,engine="openpyxl")
    fileid = values['value'].loc[values['property']=='identifier']
    version = values['value'].loc[values['property']=='dateModified']
    xwalk_dict['identifier'] = fileid
    xwalk_dict['version'] = version
    return xwalk_dict, data_file


def download_spreadsheet(parent_path,url):
    if 'xlsx' in url:
        extension = '.xlsx'
    elif 'google' in url:
        extension = '.xlsx'
    else:
        extension = '.xls'
    xwalk_dict, data_file = get_xwalk_meta(parent_path, url)
    identifier = xwalk_dict['identifier']
    if data_file != None:
        with open(os.path.join(parent_path,'crosswalks',f"{identifier}{extension}"), 'wb') as output:
            output.write(data_file)
    else:
        print('unable to parse data from provided url')

In [43]:
def get_json_version(data_file):
    tmpjson = json.load(open(os.path.join(parent_path,'jsoncrosswalks',data_file),'rb'))
    dateModified = tmpjson['dateModified']
    dateModified = tmpjson['dateModified']
    jsonversion = datetime.strptime(dateModified,"%Y-%m-%d")
    return jsonversion  

def get_xls_version(data_file):
    try:
        xwalkmeta = pd.read_excel(data_file,sheet_name='metainfo',header=0,index_col=0)
    except:
        xwalkmeta = pd.read_excel(data_file,sheet_name='metainfo',header=0,index_col=0,engine="openpyxl")
    xwalkdict = xwalkmeta.to_dict()
    xls_version = xwalkdict['value']['dateModified']
    return xls_version

def compare_versions(version1,version2):
    if version1 > version2:
        newer = version1
    else: #otherwise version 2 is either newer or the same, so just keep it
        newer = version2
    return newer

In [5]:
#### Check if the json_crosswalks folder is up-to-date
def check_crosswalks_conversion(parent_path):
    crosswalks_path = os.path.join(parent_path,'crosswalks')
    json_crosswalks_path = os.path.join(parent_path,'jsoncrosswalks')
    saved_crosswalks_list = os.listdir(crosswalks_path)
    json_crosswalks_list = os.listdir(json_crosswalks_path)
    ## For each item in crosswalks df
    for eachfile in saved_crosswalks_list:
        filename = eachfile.replace('.xls','.json').replace('.xlsx','.json')
     ### Check if the file already exists
        if filename in json_crosswalks_list:
            jsonversion = get_json_version(os.path.join(parent_path,'jsoncrosswalks',filename))
            xlsversion = get_xls_version(os.path.join(parent_path,'crosswalks',eachfile))
            ### If it does, check if the version is up-to-date
            if jsonversion < xlsversion:
                ### If the jsonversion is older than the xls version, run the conversion
                convert_a_crosswalk(crosswalks_path,eachfile,json_crosswalks_path)
        ### If it doesn't exist, run the conversion             
        else:
            convert_a_crosswalk(crosswalks_path,eachfile,json_crosswalks_path) 
            

#### Check if items in crosswalks list are in the crosswalks folder
def check_crosswalks_list(parent_path):
    try:
        crosswalksdf = pd.read_csv(os.path.join(parent_path,'crosswalkslist.txt'),delimiter='\t',header=0, parse_dates=['version'])
    except:
        crosswalksdf = pd.read_csv(os.path.join(parent_path,'crosswalkslist.txt'),delimiter='\t',header=0)
    crosswalks_path = os.path.join(parent_path,'crosswalks')
    saved_crosswalks_list = os.listdir(crosswalks_path)
    ## For each item in crosswalks df
    crosswalk_list = crosswalksdf['identifier'].tolist()
    for eachcrosswalk in crosswalk_list:
        tmp = crosswalksdf.loc[crosswalksdf['identifier']==identifier]
        txtversion = tmp.iloc[0]['version']
        txturl = tmp.iloc[0]['url']
        ### Check if the file already exists
        if (f"{identifier}.xls" in saved_crosswalks_list) or (f"{identifier}.xlsx" in saved_crosswalks_list):
            ### If it does, check if the version is up-to-date
            try:
                xlsversion = get_xls_version(os.path.join(parent_path,'crosswalks',f"{identifier}.xls"))
            except:
                xlsversion = get_xls_version(os.path.join(parent_path,'crosswalks',f"{identifier}.xlsx"))                
            newer = compare_versions(txtversion,xlsversion)
            ## if the one listed in the textfile is newer, download it
            if newer == txtversion:
                download_spreadsheet(parent_path,txturl)
        else:
            ### If it doesn't exist or is not up-to-date, download it
            download_spreadsheet(parent_path,txturl)

In [44]:
#### Given a url, check if it already exists in the crosswalks list or not:
  ## If it already exists, check the date (version)
    ### If it's newer, update the list and download it
  ## If it does not exist,
    ### Add it to the list and download it


2022-09-28 00:00:00
2022-09-28 00:00:00


In [4]:
inputurl = 'https://docs.google.com/spreadsheets/d/16kokDEZPDv_i4PFkbZGhrUHNhvRwv0M8Z6HNtZoqjbI/edit#gid=0'
#inputurl = 'https://docs.google.com/spreadsheets/d/16kokDEZPDv_i4PFkbZGhrUHNhvRwv0M8Z6HNtZoqjbI/edit?usp=sharing'
data_file = load_g_sheet_data(parent_path, inputurl)
values = pd.read_excel(BytesIO(data_file),sheet_name='metainfo',header=0,index_col=None,engine="openpyxl")
fileid = values['value'].loc[values['property']=='identifier']
version = values['value'].loc[values['property']=='dateModified']
print(fileid.iloc[0])
print(version.iloc[0])


nde_xref_bioconductor_ComputationalTools
2022-08-14 00:00:00


In [ ]:
## Save xlsx template file from a GitHub repository

In [ ]:
## Save xlsx template file given a publicly viewable google spreadsheet link

In [2]:
crosswalkstable = pd.read_csv(os.path.join(parent_path,'crosswalkslist.txt'),delimiter='\t',header=0)
print(crosswalkstable)

                              identifer  \
0  nde_xref_biotools_ComputationalTools   

                                                 url urlType    version  
0  https://github.com/gtsueng/DDE-CrossWalks/blob...  GitHub  9/28/2022  


In [7]:
githuburl = crosswalkstable.iloc[0]['GitHubUrl']
print(url)


data_file = load_github_data(parent_path,githuburl)


https://github.com/gtsueng/DDE-CrossWalks/blob/main/crosswalks/nde_biotools_computationaltool.xls


In [31]:
identifier = 'test'
url = 'https://docs.google.com/spreadsheets/d/16kokDEZPDv_i4PFkbZGhrUHNhvRwv0M8Z6HNtZoqjbI/edit#gid=0'

download_spreadsheet(parent_path,url,identifier)

In [6]:
### Convert GitHub url

'https://github.com/gtsueng/DDE-CrossWalks/raw/main/crosswalks/nde_biotools_computationaltool.xls'
'https://github.com/gtsueng/DDE-CrossWalks/blob/mygene/crosswalks/nde_biotools_computationaltool.xls?raw=true'

NameError: name 'url' is not defined